# Next Word Prediction using Tenserflow and Keras Library
# Author : Nischitha D
# Data Science Intern@Letsgrowmore

# Import Iibraries 

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the text data

In [3]:
with open('1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocessing the text , Convert to lowercase and split into words


In [4]:
corpus = text.lower().split()

# Creating tokenization

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Creating input sequences and labels

In [6]:
input_sequences = []
for i in range(1, len(corpus)):
    sequence = corpus[i-1:i+1]
    input_sequences.append(sequence)

# Convert input sequences to numeric form

In [7]:
sequences = tokenizer.texts_to_sequences(input_sequences)

#  Pad sequences to have consistent length

In [8]:
max_sequence_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

#  Split sequences into input (X) and label (y)

In [9]:
X = sequences[:, 0]
y = sequences[:, 1]

# Building RNN Model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.SimpleRNN(150),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model

In [12]:
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
3363/3363 [==============================] - 101s 30ms/step - loss: 5.8816 - accuracy: 0.1022
Epoch 2/50
3363/3363 [==============================] - 98s 29ms/step - loss: 5.2668 - accuracy: 0.1366
Epoch 3/50
3363/3363 [==============================] - 104s 31ms/step - loss: 5.0433 - accuracy: 0.1536
Epoch 4/50
3363/3363 [==============================] - 105s 31ms/step - loss: 4.8795 - accuracy: 0.1656
Epoch 5/50
3363/3363 [==============================] - 104s 31ms/step - loss: 4.7503 - accuracy: 0.1708
Epoch 6/50
3363/3363 [==============================] - 104s 31ms/step - loss: 4.6488 - accuracy: 0.1753
Epoch 7/50
3363/3363 [==============================] - 103s 31ms/step - loss: 4.5701 - accuracy: 0.1781
Epoch 8/50
3363/3363 [==============================] - 100s 30ms/step - loss: 4.5118 - accuracy: 0.1789
Epoch 9/50
3363/3363 [==============================] - 101s 30ms/step - loss: 4.4685 - accuracy: 0.1786
Epoch 10/50
3363/3363 [==============================] -

#  Generating the next word

In [26]:
def generate_next_sentence(seed_text, model, tokenizer, max_sequence_len, num_words=20):
    generated_text = seed_text
    predictions = []

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                predictions.append(output_word)
                break
        generated_text += " " + output_word

    return generated_text, predictions

In [27]:
seed_text = "It is not a lack of love, but a lack of"
generated_text, predictions = generate_next_sentence(seed_text, model, tokenizer, max_sequence_len, num_words=5)

print(seed_text)
print("Associated Predictions:", predictions)
print("Generated Text:", generated_text)

It is not a lack of love, but a lack of
Associated Predictions: ['the', 'other', 'was', 'a', 'very']
Generated Text: It is not a lack of love, but a lack of the other was a very
